In [1]:
# # Mount the Google drive onto Colab Virtual Environment
# from google.colab import drive
# drive.mount('/content/drive')
root = '.'
# root = '/content/drive/My Drive/English Dataset'

In [28]:
import pandas as pd
import numpy as np
import string
from string import digits
import matplotlib.pyplot as plt
%matplotlib inline
import re
from sklearn.utils import shuffle
from sklearn.model_selection import train_test_split
from keras.layers import Input, Embedding, Dense, CuDNNLSTM, LSTM
from keras.models import Model
import tensorflow as tf
import pickle
import os
from datetime import datetime

In [5]:
with open(os.path.join(root, 'en-mar_cleaned.en')) as f:
    lines = f.readlines()
    
lines = pd.DataFrame(lines, columns=['Src'])
lines['Tar'] = None

with open(os.path.join(root, 'en-mar_cleaned.ma')) as f:
    lines['Tar'] = f.readlines()

In [6]:
lines.Src = lines.Src.apply(lambda x: x.strip())
lines.Tar = lines.Tar.apply(lambda x: x.strip())
lines.Tar = '<START> ' + lines.Tar + ' <END>'

In [7]:
lines.shape

(33725, 2)

In [8]:
lines.sample(10)

,Src,Tar
20000,the keys are on the table,<START> चाव्या टेबलावर आहेत <END>
15399,im having lots of fun,<START> मला खूप मजा येत आहे <END>
11963,it was fun yesterday,<START> काल मजा आली <END>
32258,there are many words that i dont understand,<START> असे भरपूर शब्द आहेत जे मला कळत नाहीत <...
6934,the tank is empty,<START> टाकी रिकामी आहे <END>
31835,the building on the right side is a school,<START> उजव्या बाजूला असलेली बिल्डिंग एक शाळा ...
26584,night is when most people sleep,<START> बहुतेक लोकं रात्री झोपतात <END>
4784,give me your gun,<START> बंदूक दे <END>
24859,she stayed there for a moment,<START> ती तिथे एक क्षण राहिली <END>
17746,did you feed the parrots,<START> पोपटांना खायला दिलंस का <END>


In [9]:
all_src_words = set()
for line in lines.Src:
    all_src_words = all_src_words.union(set(list(line.split(' '))))
    
all_tar_words = set()
for line in lines.Tar:
    all_tar_words = all_tar_words.union(set(list(line.split(' '))))

In [14]:
src_dict = dict()

for index, word in enumerate(sorted(all_src_words)):
    src_dict[word] = len(src_dict)
    
tar_dict = dict()

for index, word in enumerate(sorted(all_tar_words)):
    tar_dict[word] = len(tar_dict)

In [15]:
src_rev_dict = dict()

for key, value in src_dict.items():
    src_rev_dict[value] = key
    
tar_rev_dict = dict()

for key, value in tar_dict.items():
    tar_rev_dict[value] = key

In [16]:
max_len_src = max([len(sentence.split(' ')) for sentence in lines.Src])
max_len_src

34

In [17]:
max_len_tar = max([len(sentence.split(' ')) for sentence in lines.Tar])
max_len_tar

37

In [18]:
num_encoder_tokens = len(all_src_words)
num_decoder_tokens = len(all_tar_words)
(num_encoder_tokens, num_decoder_tokens)

(5388, 12697)

In [19]:
lines = shuffle(lines)
lines.head(10)

,Src,Tar
17100,that isnt what i meant,<START> माझा तो अर्थ नव्हता <END>
2688,i was learning,<START> मी शिकत होते <END>
15443,is everybody listening,<START> सगळे ऐकताहेत का <END>
27817,i like learning ancient languages,<START> मला प्राचीन भाषा शिकायला आवडतात <END>
22374,i will buy a car next month,<START> मी पुढच्या महिन्यात एक गाडी विकत घेईन ...
20952,i have already eaten lunch,<START> मी आधीच जेवलो आहे <END>
2616,i like bananas,<START> मला केळी आवडतात <END>
10996,where were you born,<START> तुझा जन्म कुठे झाला होता <END>
25422,dont forget to write the date,<START> दिनांक लिहायला विसरू नकोस <END>
1448,tom needs me,<START> टॉमला माझी गरज आहे <END>


In [22]:
def encode_source(batch_input, batch_size):
    encoder_input_data = np.zeros((batch_size, max_len_src), dtype='float32')
    
    for i, input_text in enumerate(batch_input):
        for t, word in enumerate(input_text.split(' ')):
            if word not in src_dict.keys():
                word = 'UNK'
                
            encoder_input_data[i, t] = src_dict[word]
            
    return encoder_input_data

def encode_target(batch_output, batch_size):
    decoder_input_data = np.zeros((batch_size, max_len_tar), dtype='float32')
    decoder_target_data = np.zeros((batch_size, max_len_tar, num_decoder_tokens), dtype='float32')
    
    for i, target_text in enumerate(batch_output):
        for t, word in enumerate(target_text.split(' ')):
            if word not in tar_dict.keys():
                word = 'UNK'
                
            if t < len(target_text.split())-1:
                decoder_input_data[i, t-1] = tar_dict[word]
                
            if t > 0:
                decoder_target_data[i, t-1, tar_dict[word]] = 1
                
    return decoder_input_data, decoder_target_data

In [23]:
def generate_batch(X, y, batch_size):
    while True:
        for j in range(0, len(X), batch_size):
            encoder_input_data = encode_source(X[j:j+batch_size], batch_size)
            decoder_input_data, decoder_target_data = encode_target(y[j:j+batch_size], batch_size)
            
            yield([encoder_input_data, decoder_input_data], decoder_target_data)

In [24]:
X, y = lines.Src, lines.Tar
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.1)
X_train.shape, X_test.shape

((30352,), (3373,))

In [32]:
train_samples = len(X_train)
val_samples = len(X_test)
batch_size = 32
epochs = 50

In [25]:
latent_dim = 128

In [26]:
encoder_inputs = Input(shape=(None, ), name='Inputs')
enc_emb = Embedding(num_encoder_tokens, latent_dim, mask_zero=True, name='English_Embeddings2')(encoder_inputs)
encoder_lstm = LSTM(latent_dim, return_state=True, name='Encoder_LSTM')
encoder_outputs, state_h, state_c = encoder_lstm(enc_emb)
encoder_states = [state_h, state_c]

In [29]:
decoder_inputs = Input(shape=(None, ), name='Outputs')
dec_emb_layer = Embedding(num_decoder_tokens, latent_dim, mask_zero=True, name='Marathi_Embeddings')
dec_emb = dec_emb_layer(decoder_inputs)
decoder_lstm = LSTM(latent_dim, return_sequences=True, return_state=True, name='Decoder_LSTM')
decoder_outputs, _, _ = decoder_lstm(dec_emb, initial_state=encoder_states)
decoder_dense = Dense(num_decoder_tokens, activation='softmax', name='Dense_Eng_Mar' + str(datetime.today().date()))
decoder_outputs = decoder_dense(decoder_outputs)

In [30]:
model = Model([encoder_inputs, decoder_inputs], decoder_outputs)

In [31]:
model.compile(optimizer='rmsprop', loss='categorical_crossentropy', metrics=['acc'])

In [33]:
model.load_weights(os.path.join(root, 'translator_en_hi.h5'), by_name=True)

OSError: Unable to open file (unable to open file: name = 'translator_en_hi.h5', errno = 2, error message = 'No such file or directory', flags = 0, o_flags = 0)

In [ ]:
filepath = os.path.join(root, "best_model_en_ma.hdf5")
checkpoint = ModelCheckpoint(filepath, monitor='acc', verbose=1, save_best_only=True, mode='max')
callbacks_list = [checkpoint]

In [ ]:
model.fit_generator(generator = generate_batch(X_train, y_train, batch_size = batch_size),
                    steps_per_epoch = train_samples//batch_size,
                    epochs=epochs,
                    validation_data = generate_batch(X_test, y_test, batch_size = batch_size),
                    validation_steps = val_samples//batch_size, 
                    callbacks=callbacks_list)

In [ ]:
model.save(os.path.join(root, 'translator_en_ma.h5'))

In [ ]:
encoder_model = Model(encoder_inputs, encoder_states)

decoder_state_input_h = Input(shape=(latent_dim, ))
decoder_state_input_c = Input(shape=(latent_dim, ))
decoder_states_inputs = [decoder_state_input_h, decoder_state_input_c]

dec_emb2 = decoder_embedding_layer(decoder_inputs)

decoder_outputs2, state_h2, state_c2 = decoder_lstm(dec_emb2, initial_state=decoder_states_inputs)
decoder_states2 = [state_h2, state_c2]
decoder_outputs2 = decoder_dense(decoder_outputs2)

decoder_model = Model(
    [decoder_inputs] + decoder_states_inputs,
    [decoder_outputs2] + decoder_states2
)

In [ ]:
def decode_sequence(input_seq):
    states_value = encoder_model.predict(input_seq)
    target_seq = np.zeros((1, 1))
    target_seq[0, 0] = tar_dict['<START>']
    
    stop_condition = False
    decoded_sentence = ''
    
    while not stop_condition:
        output_tokens, h, c = decoder_model.predict([target_seq] + states_value)
        
        sampled_token_index = np.argmax(output_tokens[0, -1, :])
        sampled_char = tar_rev_dict[sampled_token_index]
        decoded_sentence += ' ' + sampled_char
        
        if (sampled_char == '<END>') or (len(decoded_sentence) > 50):
            stop_condition = True
            
        target_seq = np.zeros((1, 1))
        target_seq[0, 0] = sampled_token_index
        
        states_value = [h, c]
    return decoded_sentence

In [0]:
train_gen = generate_batch(X_train, y_train, batch_size = 1)
k=-1

In [37]:
for i in range(10):
    k+=1
    (input_seq, actual_output), _ = next(train_gen)
    decoded_sentence = decode_sequence(input_seq)
    print('Input English sentence:', X_train[k:k+1].values[0])
    print('Actual Marathi Translation:', y_train[k:k+1].values[0][7:-5])
    print('Predicted Marathi Translation:', decoded_sentence[:-5])

Input English sentence: col lection
Actual Hindi Translation:  संग्रह 
Predicted Hindi Translation:  विवरण विवरण 


In [0]:
import json

with open("/var/log/colab-jupyter.log", "r") as fo:
  for line in fo:
    print(json.loads(line)['msg'])

Writing notebook server cookie secret to /root/.local/share/jupyter/runtime/notebook_cookie_secret
google.colab serverextension initialized.
Serving notebooks from local directory: /
0 active kernels
The Jupyter Notebook is running at:
http://172.28.0.2:9000/
Use Control-C to stop this server and shut down all kernels (twice to skip confirmation).
Kernel started: c4dbbf68-7cc1-4958-81c9-84080122875b
Adapting to protocol v5.1 for kernel c4dbbf68-7cc1-4958-81c9-84080122875b
Adapting to protocol v5.1 for kernel c4dbbf68-7cc1-4958-81c9-84080122875b
2019-04-01 14:36:25.486024: I tensorflow/core/platform/profile_utils/cpu_utils.cc:94] CPU Frequency: 2300000000 Hz
2019-04-01 14:36:25.486495: I tensorflow/compiler/xla/service/service.cc:150] XLA service 0x19c2520 executing computations on platform Host. Devices:
2019-04-01 14:36:25.486530: I tensorflow/compiler/xla/service/service.cc:158]   StreamExecutor device (0): <undefined>, <undefined>
2019-04-01 14:36:25.633812: I tensorflow/stream_exec